In [21]:
from IPython.display import Image

In [22]:
'''
--Imports--
BeautifulSoup is used to parse through Html data.
Requests gets us the data from the webpage.
csv is used to output the data we get into a csv.

'''
from bs4 import BeautifulSoup as bs
import requests
import csv


In [23]:
# This is standard in every website you will do this. Request and use BeautifulSoup to "Soupify". 
# The output is the code for the whole page

url = 'https://member.usafencing.org/clubs'
req = requests.get(url)
soup = bs(req.content)
# soup.tbody.tr
# tr = soup.tbody.find_all('tr')
# tr
soup

<!DOCTYPE html>
<html lang="en"><head>


    <!-- Begin Head Code -->
<!-- Required meta tags always come first -->
<meta charset="utf-8"/>
<title>Clubs - USA Fencing</title>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>

<link href="/favicon.ico?v=20.8.15" rel="shortcut icon" type="image/x-icon"/>


<link href="//fast.fonts.net/cssapi/fcd3626c-96e9-4a56-8d2b-db5ed49104e0.css" rel="stylesheet" type="text/css"/>
<link href="/css/dsxl.css?v=20.8.15" rel="stylesheet"/>

<script src="/js/head.js?v=20.8.15"></script>

<!-- End Head Code -->

    <!-- Extra Head Code -->
    
</head>
<body class="">
    <!-- LAYOUT.BLADE.PHP -->

    <div class="TopBar" id="TopBar">
    <div class="container-fluid">
        <div class="row">
            <div class="col-24">

                <div class="nav-aligner d-flex justify-content-between align-items-center">


                    <a class="logo" href="http

In [24]:
#this will get you all the information in the tag
name1= soup.find(class_='lead mb-0') 
name1


<p class="lead mb-0">"A' La" Fencing Instruction (AFI)</p>

In [25]:
#this just the text, in some cases you will need to do some cleaning. We will do it later.
name1.text

'"A\' La" Fencing Instruction (AFI)'

In [26]:
#find all will find all elements in the page with that tag and return a list with all of them
name_all = soup.find_all(class_='lead mb-0') 
name_all

[<p class="lead mb-0">"A' La" Fencing Instruction (AFI)</p>,
 <p class="lead mb-0">5T Fencers Club (5TFC)</p>,
 <p class="lead mb-0">Abilene Sport Fencing (ASF)</p>,
 <p class="lead mb-0">Academies Of Fencing In Baltimore (AOFIB)</p>,
 <p class="lead mb-0">Academy Of Fencing Masters (AFM) (AFM)</p>,
 <p class="lead mb-0">Ace Fencing Academy (ACE)</p>,
 <p class="lead mb-0">Advance Fencing Academy (Salle Stepien) (AFA-SS)</p>,
 <p class="lead mb-0">Advance Fencing And Fitness Academy (AFFA)</p>,
 <p class="lead mb-0">AIC  Fencing Club (AIC)</p>,
 <p class="lead mb-0">AK SE Fencers (AKSEF)</p>]

In [27]:
address1 = soup.find(class_='address tight')
address1

<p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                10 Burgess Place<br/>Palm Coast, FL 32127, US
            </p>

In [28]:
address1.text

'\n                10 Burgess PlacePalm Coast, FL 32127, US\n            '

In [29]:
# to get rid of the special spacing characters.
address1.text.strip()

'10 Burgess PlacePalm Coast, FL 32127, US'

In [30]:
# We can also use find all here
address_all = soup.find_all(class_='address tight')
address_all

[<p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                 10 Burgess Place<br/>Palm Coast, FL 32127, US
             </p>,
 <p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                 63 East 2nd Street<br/>Mineola, NY 11501, US
             </p>,
 <p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                 1174 Butternut St<br/>Abilene, TX 79602, US
             </p>,
 <p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                 1120 N Rolling Road<br/>Catonsville, MD 21228, US
             </p>,
 <p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                

In [31]:
# Each club is in a "tr" tag this is what it looks like:
soup.find_all('tr')[1]

<tr>
                                                                                                                                                                                                                                                                                                                                                                        <td colspan="2">
        <p class="lead mb-0">"A' La" Fencing Instruction (AFI)</p>
         <p class="mb-0"><strong>Pending Renewal</strong></p>         <p><span class="text-muted"> #100025092, Jul 31, 2022</span></p>
    </td>

    <td>

                        <p class="address tight" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress" style="margin-bottom:.5rem;">
                10 Burgess Place<br/>Palm Coast, FL 32127, US
            </p>
            
                                    <p class="tight" style="margin-bottom:.5rem;">(386) 793-3142</p>
                
                               

In [32]:
def find_name(soup):
    text = soup.find(class_='lead mb-0').text.strip()
    index = text.find('(') # find the first instance of '(' because we only want the name for now
    if index == -1:
        return text
    return text[:index -1]    

In [33]:
def find_address(soup):
    if soup.find(class_='address tight') is not None:
        text = soup.find(class_='address tight').get_text(separator=" ").strip()
        if text == '':
            return None
        return text
    return None

In [35]:
def find_city(soup):
    '''
    For city the format we use in the database is always City - Two letter acronym for state. It is important that cities
    always follow this format. Example:
    Los Angeles - CA
    New York - NY
    Baltimore - MD
    '''
    if soup.find(class_='address tight'):
        text = soup.find(class_='address tight').get_text(separator="||").strip()
        ind = text.find("||")
        if ind == -1:
            return None
        res = text[ind + 2:].split(',')
        return res[0] + ' -' + res[1][:3]
    return None

In [36]:
def find_state(city):
    if city is None:
        return None
    return city[-2:]

In [37]:
def find_phone(soup):
    all_data = soup.find_all(class_='tight')
    for data in all_data:
        if len(data.text) > 0:
            if data.text[0] == '(':    # this line checks if its a phone number
                return data.text
    return None

In [38]:
def find_email(soup):
    all_data = soup.find_all(class_='tight')
    for data in all_data:
        if '@' in data.text:
            return data.text
    return None

In [39]:
def find_website(soup):
    all_data = soup.find_all(class_='tight')
    for data in all_data:
        if soup.find('a'):
            return soup.find('a')['href']
    return None

In [40]:
def find_acronym(soup):
    text = soup.find(class_='lead mb-0').text.strip()
    index = text.find('(') # find the first instance of '(' because we only want the name for now
    if index == -1:
        return None
    return text[index + 1:-1]

In [42]:
def main():
    #this list will store every dictionary of each club
    my_list = []
    
    #used for iterating over pages. There are 60 pages of data  
    for i in range (1,61):   #1 to 61
        print(i)
       
        #requesting the data
        url = 'https://member.usafencing.org/clubs?page=' + str(i)
        req = requests.get(url)
        soup = bs(req.content)
        
        # Every club is in a "tr" tag. So I will separate each club first to make it easier
        all_clubs = soup.find_all('tr')
        
        for row in all_clubs[1:]:
            # This dict will store the information of each club
            my_dict = {}
            
            my_dict['name'] = find_name(row)
            my_dict['acronym']= find_acronym(row)
            my_dict['address'] = find_address(row)
            my_dict['city'] = find_city(row)
            my_dict['state'] = find_state(my_dict['city'])
            my_dict['phone'] = find_phone(row)
            my_dict['email'] = find_email(row)
            my_dict['website'] = find_website(row)
            
            #this are all the constants that must be added
            my_dict['country'] = 'United States of America'
            my_dict['Affilated'] = 'USA Fencing'
            my_dict['Recorded By'] = 'Cezar Pekelman'
            my_dict['Recorded Date'] = '10/10/2021'

            my_list.append(my_dict)
            
        
        i = i + 1
    
    return my_list
        
res = main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [44]:
# this is what the output looks like as a list of dictionaries
res

[{'name': '"A\' La" Fencing Instruction',
  'acronym': 'AFI',
  'address': '10 Burgess Place Palm Coast, FL 32127, US',
  'city': 'Palm Coast - FL',
  'state': 'FL',
  'phone': '(386) 793-3142',
  'email': 'coacht@alafencing.net',
  'website': 'http://www.alafencing.net',
  'country': 'United States of America',
  'Affilated': 'USA Fencing',
  'Recorded By': 'Cezar Pekelman',
  'Recorded Date': '10/10/2021'},
 {'name': '5T Fencers Club',
  'acronym': '5TFC',
  'address': '63 East 2nd Street Mineola, NY 11501, US',
  'city': 'Mineola - NY',
  'state': 'NY',
  'phone': '(516) 295-4148',
  'email': '5tfencersclub@gmail.com',
  'website': 'http://5TFencersclub.com',
  'country': 'United States of America',
  'Affilated': 'USA Fencing',
  'Recorded By': 'Cezar Pekelman',
  'Recorded Date': '10/10/2021'},
 {'name': 'Abilene Sport Fencing',
  'acronym': 'ASF',
  'address': '1174 Butternut St Abilene, TX 79602, US',
  'city': 'Abilene - TX',
  'state': 'TX',
  'phone': '(325) 660-9061',
  'ema

In [46]:
#Exporting to csv
keys = res[0].keys()


with open('US_fencing.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = keys
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(res)
    
csvfile.close()